Halite is an online multiplayer game created by Two Sigma.  In the game, four participants command ships to collect an energy source called **halite**.  The player with the most halite at the end of the game wins.

In this tutorial, as part of the **[Halite competition](https://www.kaggle.com/c/halite/overview)**, you'll write your own intelligent bots to play the game.  

> Note that the Halite competition is now closed, so we are no longer accepting submissions.  That said, you can still use the competition to write your own bots - you just cannot submit bots to the official leaderboard.  To see the current list of open competitions, check out the simulations homepage: https://www.kaggle.com/simulations.


# Part 1: Get started

In this section, you'll learn more about how to play the game. 

## Game rules 

In this section, we'll look more closely at the game rules and explore the different icons on the game board.

For context, we'll look at a game played by four random players.  You can use the animation below to view the game in detail: every move is captured and can be replayed. 

In [ ]:
#$HIDE_INPUT$
from kaggle_environments import make, evaluate
env = make("halite", debug=True)
env.run(["random", "random", "random", "random"])
env.render(mode="ipython", width=800, height=600)

The game is played in a 21 by 21 gridworld and lasts 400 timesteps.  Each player starts the game with 5,000 halite and one ship.  

Grid locations with **halite** are indicated by a light blue icon, where larger icons indicate more available halite.

<center>
<img src="https://storage.googleapis.com/kaggle-media/learn/images/3NENMos.png" width=65%><br/>
</center>

Players use **ships** to navigate the world and collect halite.  A ship can only collect halite from its current position.  When a ship decides to collect halite, it collects 25% of the halite available in its cell.  This collected halite is added to the ship's "cargo".  

<center>
<img src="https://storage.googleapis.com/kaggle-media/learn/images/eKN0kP3.png" width=65%><br/>
</center>

Halite in ship cargo is not counted towards final scores.  In order for halite to be counted, ships need to deposit their cargo into a **shipyard** of the same color.  A ship can deposit all of its cargo in a single timestep simply by navigating to a cell containing a shipyard.

<center>
<img src="https://storage.googleapis.com/kaggle-media/learn/images/LAc6fj8.png" width=65%><br/>
</center>

Players start the game with no shipyards.  To get a shipyard, a player must convert a ship into a shipyard, which costs 500 halite.  Also, shipyards can spawn (or create) new ships, which deducts 500 halite (per ship) from the player.

Two ships cannot successfully inhabit the same cell.  This event results in a **collision**, where:
- the ship with more halite in its cargo is destroyed, and 
- the other ship survives and instantly collects the destroyed ship's cargo.

<center>
<img src="https://storage.googleapis.com/kaggle-media/learn/images/BuIUPmK.png" width=65%><br/>
</center>

If you view the full game rules, you'll notice that there are more types of collisions that can occur in the game (for instance, ships can collide with enemy shipyards, which destroys the ship, the ship's cargo, and the enemy shipyard). 

In general, Halite is a complex game, and we have not covered all of the details here.  But even given these simplified rules, you can imagine that a successful player will have to use a relatively complicated strategy.  

## Game strategy

As mentioned above, a ship has two options at its disposal for collecting halite.  It can:
- collect (or mine) halite from its current position.
- collide with an enemy ship containing relatively more halite in its cargo.  In this case, the ship destroys the enemy ship and steals its cargo.

Both are illustrated in the figure below.  The "cargo" that is tracked in the player's scoreboard contains the total cargo, summed over all of the player's ships.
<center>
<img src="https://storage.googleapis.com/kaggle-media/learn/images/2DJX6Vt.png" width=75%><br/>
</center>

This raises some questions that you'll have to answer when commanding ships:
- Will your ships focus primarily on locating large halite reserves and mining them efficiently, while mostly ignoring and evading the other players?  
- Or, will you look for opportunities to steal halite from other players?  
- Alternatively, can you use a combination of those two strategies?  If so, what cues will you look for in the game to decide which option is best?  For instance, if all enemy ships are far away and your ships are located on cells containing a lot of halite, it makes sense to focus on mining halite.  Conversely, if there are many ships nearby with halite to steal (and not too much local halite to collect), it makes sense to attack the enemy ships.

You'll also have to decide how to control your shipyards, and how your ships interact with shipyards.  There are three primary actions in the game involving shipyards.  You can:
- convert a ship into a shipyard.  This is the only way to create a shipyard.  
- use a shipyard to create a ship.
- deposit a ship's cargo into a shipyard. 

These are illustrated in the image below.

<center>
<img src="https://storage.googleapis.com/kaggle-media/learn/images/fL5atut.png" width=75%><br/>
</center>

With more ships and shipyards, you can collect halite at a faster rate.  But each additional ship and shipyard costs you halite: how will you decide when it might be beneficial to create more?

# Part 2: Your first bot

In this section, you'll create your first bot to play the game.

## The notebook

The first thing to do is to create a Kaggle notebook where you'll store all of your code. 

Begin by navigating to [**https://www.kaggle.com/notebooks**](https://www.kaggle.com/notebooks) and clicking on **"New Notebook"**.

Next, click on **"Create"**. (_Don't change the default settings: so, **"Python"** should appear under **"Select language"**, and you should have **"Notebook"** selected under **"Select type"**._)

![](https://storage.googleapis.com/kaggle-media/learn/images/qUVvr8k.png)

You now have a notebook where you'll develop your first agent!  If you're not sure how to use Kaggle Notebooks, we strongly recommend that you walk through **[this notebook](https://www.kaggle.com/alexisbcook/getting-started-with-titanic)** before proceeding.  It teaches you how to run code in the notebook.

## Your first agent

It's time to create your first agent!  Copy and paste the code in the cell below into your notebook.  Then, run the code.

In [ ]:
%%writefile submission.py

# Imports helper functions
from kaggle_environments.envs.halite.helpers import *

# Returns best direction to move from one position (fromPos) to another (toPos)
# Example: If I'm at pos 0 and want to get to pos 55, which direction should I choose?
def getDirTo(fromPos, toPos, size):
    fromX, fromY = divmod(fromPos[0],size), divmod(fromPos[1],size)
    toX, toY = divmod(toPos[0],size), divmod(toPos[1],size)
    if fromY < toY: return ShipAction.NORTH
    if fromY > toY: return ShipAction.SOUTH
    if fromX < toX: return ShipAction.EAST
    if fromX > toX: return ShipAction.WEST

# Directions a ship can move
directions = [ShipAction.NORTH, ShipAction.EAST, ShipAction.SOUTH, ShipAction.WEST]

# Will keep track of whether a ship is collecting halite or carrying cargo to a shipyard
ship_states = {}

# Returns the commands we send to our ships and shipyards
def agent(obs, config):
    size = config.size
    board = Board(obs, config)
    me = board.current_player

    # If there are no ships, use first shipyard to spawn a ship.
    if len(me.ships) == 0 and len(me.shipyards) > 0:
        me.shipyards[0].next_action = ShipyardAction.SPAWN

    # If there are no shipyards, convert first ship into shipyard.
    if len(me.shipyards) == 0 and len(me.ships) > 0:
        me.ships[0].next_action = ShipAction.CONVERT
    
    for ship in me.ships:
        if ship.next_action == None:
            
            ### Part 1: Set the ship's state 
            if ship.halite < 200: # If cargo is too low, collect halite
                ship_states[ship.id] = "COLLECT"
            if ship.halite > 500: # If cargo gets very big, deposit halite
                ship_states[ship.id] = "DEPOSIT"
                
            ### Part 2: Use the ship's state to select an action
            if ship_states[ship.id] == "COLLECT":
                # If halite at current location running low, 
                # move to the adjacent square containing the most halite
                if ship.cell.halite < 100:
                    neighbors = [ship.cell.north.halite, ship.cell.east.halite, 
                                 ship.cell.south.halite, ship.cell.west.halite]
                    best = max(range(len(neighbors)), key=neighbors.__getitem__)
                    ship.next_action = directions[best]
            if ship_states[ship.id] == "DEPOSIT":
                # Move towards shipyard to deposit cargo
                direction = getDirTo(ship.position, me.shipyards[0].position, size)
                if direction: ship.next_action = direction
                
    return me.next_actions

The line `%%writefile submission.py` saves the agent to a Python file.  Note that all of the code above has to be copied and run in a single cell (please do not split the code into multiple cells).

If the code cell runs successfully, then you'll see a message `Writing submission.py` (or `Overwriting submission.py`, if you run it more than once).

Then, copy and run the next code cell in your notebook to play your agent against three random agents.  Your agent is in the top left corner of the screen.

In [ ]:
from kaggle_environments import make
env = make("halite", debug=True)
env.run(["submission.py", "random", "random", "random"])
env.render(mode="ipython", width=800, height=600)

Your agent should perform much better than the random agents!  Note that if your ship doesn't move at all in the game, then you've likely made an error in the code you used to generate the **submission.py** file.  If this is the case, try copying the code cell and running it again.

# Part 3: Keep learning!

This first agent is just the beginning.  Over time and with effort, your score will gradually improve.  Here are some resources to keep learning:

- The first step is to read the **[complete game rules](https://www.kaggle.com/c/halite/overview/environment-rules)**.  For simplicity, we've only covered a subset of the rules here.  But, you'll need a complete understanding of the game dynamics to come up with a winning strategy.
- The agent you just submitted uses Sam Harris' **[Halite SDK](https://www.kaggle.com/sam/halite-sdk-overview)** to simplify the code we need to write.  Check out the notebook to see what other helper functions are available.
- Look at the **[Notebooks tab](https://www.kaggle.com/c/halite/notebooks?sortBy=voteCount&group=everyone&pageSize=20&competitionId=18011)** in the Halite competition to learn from agents that other users have created.  
- Kaggle's **[Intro to Game AI and Reinforcement Learning](https://www.kaggle.com/learn/intro-to-game-ai-and-reinforcement-learning)** course is a great resource to get started with techniques for creating intelligent agents.